The main idea here is that I want to guess what are the most important keys ahead of time ()

In [1]:
# For boba

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

import torch
print(torch.cuda.device_count())

2


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16, attn_implementation='sdpa', device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

/home/vijay/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from typing import Optional, Tuple
from transformers.cache_utils import Cache
from transformers.models.llama.modeling_llama import apply_rotary_pos_emb, repeat_kv
import math
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from time import perf_counter

import numpy as np
import torch
import triton
import triton.language as tl

In [14]:
def sdpa_forward(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    position_ids: Optional[torch.LongTensor] = None,
    past_key_value: Optional[Cache] = None,
    output_attentions: bool = False,
    use_cache: bool = False,
) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
    # print("Running custom forward function")
    bsz, q_len, _ = hidden_states.size()

    query_states = self.q_proj(hidden_states)
    key_states = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)

    query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

    kv_seq_len = key_states.shape[-2]
    if past_key_value is not None:
        kv_seq_len += past_key_value.get_usable_length(kv_seq_len, self.layer_idx)
    cos, sin = self.rotary_emb(value_states, seq_len=kv_seq_len)

    query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin, position_ids)

    if past_key_value is not None:
        cache_kwargs = {"sin": sin, "cos": cos}  # Specific to RoPE models
        key_states, value_states = past_key_value.update(key_states, value_states, self.layer_idx, cache_kwargs)

    key_states = repeat_kv(key_states, self.num_key_value_groups)
    value_states = repeat_kv(value_states, self.num_key_value_groups)

    if attention_mask is not None:
        if attention_mask.size() != (bsz, 1, q_len, kv_seq_len):
            raise ValueError(
                f"Attention mask should be of size {(bsz, 1, q_len, kv_seq_len)}, but is {attention_mask.size()}"
            )

    # SDPA with memory-efficient backend is currently (torch==2.1.2) bugged with non-contiguous inputs with custom attn_mask,
    # Reference: https://github.com/pytorch/pytorch/issues/112577.
    if query_states.device.type == "cuda" and attention_mask is not None:
        query_states = query_states.contiguous()
        key_states = key_states.contiguous()
        value_states = value_states.contiguous()

    # generate weights for which tokens are important or not

    # Apply padding to q and k
    max_block_size = 16 * 16 # apparently this is the only way that it's allowed. 
    k_len = key_states.shape[-2]
    pad_len_q = 0 if (q_len % max_block_size) == 0 else (max_block_size - (q_len % max_block_size))
    pad_len_k = 0 if (k_len % max_block_size) == 0 else (max_block_size - (k_len % max_block_size))
    if pad_len_q > 0:
        bq, hq, tq, dq = query_states.shape
        pad_matrix = torch.zeros((bq, hq, pad_len_q, dq), device=query_states.device)
        query_states = torch.cat((query_states, pad_matrix), dim=-2)  # Padding on the last dimension
    if pad_len_k > 0:
        bk, hk, tk, dk = key_states.shape
        pad_matrix = torch.zeros((bk, hk, pad_len_k, dk), device=query_states.device)
        key_states = torch.cat((key_states, pad_matrix), dim=-2)  # Padding on the last dimension
        value_states = torch.cat((value_states, pad_matrix), dim=-2)
    print("Padding length: ", pad_len_q, pad_len_k, "Q states: ", query_states.shape, " Key states: ", key_states.shape)

    # attn_output = torch.nn.functional.scaled_dot_product_attention(
    #     q_blocks,
    #     k_blocks,
    #     v_blocks,
    #     attn_mask=attention_mask,
    #     dropout_p=self.attention_dropout if self.training else 0.0,
    #     # The q_len > 1 is necessary to match with AttentionMaskConverter.to_causal_4d that does not create a causal mask in case q_len == 1.
    #     is_causal=self.is_causal and attention_mask is None and q_len > 1,
    # )

    # TODO: rejoin the output to fit the desired size.
    output = torch.empty_like(query_states)
    for head in range(query_states.size(1)):
        q = query_states[:, head, :, :].unsqueeze(1)
        k = key_states[:, head, :, :].unsqueeze(1)
        v = value_states[:, head, :, :].unsqueeze(1)
        # if self.layer_idx >= self.starting_layer and not self.is_search:
        attn_output = block_sparse_attention(q, k, v, head, block_size_M=16, block_size_N=16, subselect_scale=16) 
        # else:
        #     attn_output = torch.nn.functional.scaled_dot_product_attention(
        #         q,
        #         k,
        #         v,
        #         attn_mask=attention_mask,
        #         is_causal=self.is_causal
        #     )
        output[:, head:head + 1] = attn_output

    attn_output = attn_output.transpose(1, 2).contiguous()
    attn_output = attn_output.reshape(bsz, q_len, self.hidden_size) # q_len is padded to be of the same length for everything

    attn_output = self.o_proj(attn_output)

    return attn_output, None, past_key_value

for i in range(len(model.model.layers)):
    model.model.layers[i].self_attn.forward = sdpa_forward.__get__(model.model.layers[i].self_attn, type(model.model.layers[i].self_attn))

In [15]:
sample_text = open("snapkv_full.txt", "r", encoding="utf-8").read()
encoded_tokens = tokenizer(sample_text, return_tensors="pt")
for key in encoded_tokens:
    encoded_tokens[key] = encoded_tokens[key].cuda()

# find a way to start by chunking this portion

# there shouuld be a mix between the global attention and the attention that only the last few tokens (query) pay

past_key_values = None
outputs = model.generate(**encoded_tokens, output_attentions=True, return_dict_in_generate=True, max_new_tokens=1)

Padding length:  79 79 Q states:  torch.Size([1, 32, 17664, 128])  Key states:  torch.Size([1, 32, 17664, 128])
Params: block_m 16 block_n 16 subselect_scale 16
Query shape in _build_block_index:  torch.Size([1, 1, 17664, 128])  Key shape in _build_block_index:  torch.Size([1, 1, 17664, 128])
After taking the mean: query shape:  torch.Size([1, 1, 1104, 128])  key shape:  torch.Size([1, 1, 1104, 128])
Top k:  0
Block index shape:  torch.Size([1, 1, 1104, 0])


CompilationError: at 68:14:
        cols = start_n + offs_n
        # -- load k, v --
        k = tl.load(k_ptrs + cols[None, :] * stride_kn)
        v = tl.load(v_ptrs + cols[:, None] * stride_vn)
        # -- compute qk --
        qk = tl.zeros([BLOCK_M, BLOCK_N], dtype=tl.float32)
        # if start_n + BLOCK_N < seqlen:
        #     qk = tl.where(m_mask, qk, float("-inf"))
        # else:
        causal_mask = cols[None, :] <= offs_m[:, None]
        qk = tl.where(m_mask & causal_mask, qk, float("-inf"))
        qk += tl.dot(q, k)
              ^